<a href="https://colab.research.google.com/github/sasankmindtree/Intel_openvino/blob/master/TLwithVgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [176]:
!unzip train_data_1.zip

Streaming output truncated to the last 5000 lines.
 extracting: train_data_1/55.png     
 extracting: train_data_1/550.png    
 extracting: train_data_1/5500.png   
 extracting: train_data_1/5501.png   
 extracting: train_data_1/5502.png   
 extracting: train_data_1/5503.png   
 extracting: train_data_1/5504.png   
 extracting: train_data_1/5505.png   
 extracting: train_data_1/5506.png   
 extracting: train_data_1/5507.png   
 extracting: train_data_1/5508.png   
 extracting: train_data_1/5509.png   
 extracting: train_data_1/551.png    
 extracting: train_data_1/5510.png   
 extracting: train_data_1/5511.png   
 extracting: train_data_1/5512.png   
 extracting: train_data_1/5513.png   
 extracting: train_data_1/5514.png   
 extracting: train_data_1/5515.png   
 extracting: train_data_1/5516.png   
 extracting: train_data_1/5517.png   
 extracting: train_data_1/5518.png   
 extracting: train_data_1/5519.png   
 extracting: train_data_1/552.png    
 extracting: train_data_1/5520.png   

In [208]:
!unzip test_data_folder.zip

Archive:  test_data_folder.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of test_data_folder.zip or
        test_data_folder.zip.zip, and cannot find test_data_folder.zip.ZIP, period.


In [209]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf

In [210]:
from tensorflow.keras import Sequential

In [211]:
from keras_preprocessing.image import ImageDataGenerator

In [212]:
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

In [213]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [214]:
from tensorflow.keras import regularizers, optimizers

In [215]:
traindf=pd.read_csv(r"/content/trainLabels.csv",dtype=str,encoding = "latin-1")
testdf=pd.read_csv(r"/content/test_data.csv",dtype=str,encoding = "latin-1")

In [216]:
traindf.shape

(50000, 2)

In [217]:
testdf.shape

(1365, 2)

In [218]:
def append_ext(fn):
    return fn+".png"

def append_extr(fnr):
    return fnr+".jpg"

In [219]:
traindf["id"]=traindf["id"].apply(append_ext)
testdf["index"] = testdf["index"].apply(append_extr)

In [220]:
traindf.head()

,id,label
0,1.png,frog
1,2.png,truck
2,3.png,truck
3,4.png,deer
4,5.png,automobile


In [221]:
testdf.head()

,index,prediction
0,1.jpg,NaN
1,2.jpg,NaN
2,3.jpg,NaN
3,4.jpg,NaN
4,5.jpg,NaN


In [234]:
from tensorflow.keras.applications.vgg16 import VGG16 ,preprocess_input

In [235]:
datagen=ImageDataGenerator(preprocessing_function = preprocess_input, validation_split=0.10)

In [236]:
datagen

In [237]:
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=r"/content/train_data_1",
x_col="id",
y_col="label",
subset="training", # that means which subset we r using that will be validation or either training here we r using training
batch_size=90, #size of the batches of data (default: 32).
seed=42, #optional random seed for shuffling and transformations.
shuffle=True, # whether to shuffle the data (default: True)
class_mode="categorical",   # One of "categorical", "binary", "sparse", "input", or None. Default: "categorical". 
                            # Determines the type of label arrays that are returned: - "categorical" 
                            # will be 2D one-hot encoded labels, 
target_size=(64,64)) # Size of image Data

Found 45000 validated image filenames belonging to 10 classes.


In [238]:
train_generator

In [239]:
valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory=r"/content/train_data_1",
x_col="id",
y_col="label",
subset="validation",
batch_size=50,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(64,64))

Found 5000 validated image filenames belonging to 10 classes.


In [240]:
valid_generator

In [241]:
test_datagen=ImageDataGenerator(preprocessing_function = preprocess_input)

In [242]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory= r"/content/test_data_folder",
x_col="index",
y_col=None,
batch_size= 15,
seed=42,
shuffle=False,
class_mode=None,
validate_filenames=False,
target_size=(64,64))

Found 1365 non-validated image filenames.


In [243]:
test_generator

In [232]:
from tensorflow.keras import Sequential

In [233]:
model = Sequential()

In [244]:
model.add(VGG16(input_shape=(64,64,3),include_top = False,weights='imagenet',pooling="avg"))
for layers in model.layers:
    layers.trainable = False
model.add(Dense(10,activation = "softmax"))

58892288/58889256 [==============================] - 2s 0us/step


In [245]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [246]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [247]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [248]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
500/500 [==============================] - 46s 92ms/step - loss: 3.4825 - accuracy: 0.6302 - val_loss: 1.6638 - val_accuracy: 0.7292
Epoch 2/10
500/500 [==============================] - 45s 91ms/step - loss: 1.2627 - accuracy: 0.7522 - val_loss: 1.0907 - val_accuracy: 0.7578
Epoch 3/10
500/500 [==============================] - 45s 91ms/step - loss: 0.9067 - accuracy: 0.7723 - val_loss: 0.9300 - val_accuracy: 0.7634
Epoch 4/10
500/500 [==============================] - 45s 91ms/step - loss: 0.7831 - accuracy: 0.7799 - val_loss: 0.8027 - val_accuracy: 0.7768
Epoch 5/10
500/500 [==============================] - 45s 91ms/step - loss: 0.7459 - accuracy: 0.7811 - val_loss: 0.7863 - val_accuracy: 0.7800
Epoch 6/10
500/500 [==============================] - 45s 91ms/step - loss: 0.7220 - accuracy: 0.7852 - val_loss: 0.7824 - val_accuracy: 0.7742
Epoch 7/10
500/500 [==============================] - 45s 91ms/step - loss: 0.7172 - accuracy: 0.7856 - val_loss: 0.8452 - val_accuracy:

In [249]:
model.evaluate_generator(generator=valid_generator,steps=STEP_SIZE_VALID)

[0.8230968713760376, 0.7680000066757202]

In [250]:
test_generator.reset()

In [251]:
x = model.save('model_file.h5')

In [252]:
new_model = tf.keras.models.load_model('model_file.h5')

In [253]:
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 14,719,818
Trainable params: 5,130
Non-trainable params: 14,714,688
_________________________________________________________________


In [254]:
from PIL import Image

In [255]:
from tensorflow import keras

In [259]:
img = Image.open('/content/dog2.jpg')
img.show()

In [260]:
img = img.resize((64,64),Image.ANTIALIAS)
img_array = np.array(img)
img_array = img_array.astype('float32')
img_array /= 255
img_array = np.expand_dims(img_array,axis=0)
predict = new_model.predict(img_array)
predict

array([[0.13344133, 0.0308853 , 0.11647405, 0.12925169, 0.11804348,
        0.09619771, 0.0769207 , 0.06459814, 0.19390215, 0.04028545]],
      dtype=float32)

In [261]:
labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
index = (list(predict[0])).index(max((list(predict[0]))))
print("prediction is ---------------------->:",labels[index])

prediction is ---------------------->: Ship
